In [2]:
push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../CSDA/")

7-element Array{String,1}:
 "@"
 "@v#.#"
 "@stdlib"
 "."
 "../CSDA/"
 "."
 "../CSDA/"

In [3]:
nombre="/home/karel/ElectroNeuroTools/JLDFiles/Deshinibida/180116s1/180116s1cut_evento_1_bandascsd.h5"

"/home/karel/ElectroNeuroTools/JLDFiles/Deshinibida/180116s1/180116s1cut_evento_1_bandascsd.h5"

In [4]:
using HDF5
using Statistics
using CentrosdeMasa
using LindenbergOperadores


┌ Info: Precompiling CentrosdeMasa [top-level]
└ @ Base loading.jl:1260


In [7]:
arx=h5open(nombre)
nomine=names(arx)

5-element Array{String,1}:
 "alpha"
 "beta"
 "gamma"
 "spr"
 "uspr"

In [8]:
csds=Dict{String, Array}()

for n in nomine
    csds[n]=read(arx[n])
end

In [10]:
(ancho,alto,nmax)=size(csds["alpha"])



(64, 64, 2001)

In [14]:
cmdict=Dict{String, Any}()
factor=1.0

for n in nomine
    (mincsd,maxcsd)=extrema(csds[n])
#Esto de abajo no sirve para actividad evocada si incluimos los tiempos en que todo esta saturado
    ϵ=max(abs(mincsd),abs(maxcsd))/100
    scsd=std(csds[n])
    ##Prueba con componentes mas separados
    ##factor=5
    #aun más separados
    #factor=10
    # sin mucha separacion
    factor=1.0
    umbrsep=factor*scsd
    umbrsepnota=round(umbrsep, digits=4)
    println("empezando  la banda $n: la stddev es $scsd.")
    flush(stdout)
    cmdict[n]=ObtenComponentesyCM(csds[n],1,nmax,umbrsep);
    println("terminando  la banda $n")
    flush(stdout)
end

empezando  la banda alpha: la stddev es 1.1647684807600354.
terminando  la banda alpha
empezando  la banda beta: la stddev es 0.9718746887056936.
terminando  la banda beta
empezando  la banda gamma: la stddev es 2.147810592267772.
terminando  la banda gamma
empezando  la banda spr: la stddev es 3.916119092967827.
terminando  la banda spr
empezando  la banda uspr: la stddev es 3.916119092967827.
terminando  la banda uspr


In [17]:
nota="Se separaron los componentes con $factor desv. est. del CSD suavizado gaussiano espacial por bandas.
Cada entrada del archivo es una tupla con un componente positivo y uno negativo."

"Se separaron los componentes con 1.0 desv. est. del CSD suavizado gaussiano espacial por bandas.\nCada entrada del archivo es una tupla con un componente positivo y uno negativo."

In [18]:
# pues siempre hay que usar JLD de todas formas!
using JLD

In [19]:
nomCM=nombre[1:end-4]*"-CM.jld"

jldopen(nomCM, "w") do file
    for n in nomine
        write(file, n, cmdict[n])
    end
end

In [23]:
typeof(cmdict["alpha"][1])

Dict{Int64,Array}

In [24]:
cmposalpha=cmdict["alpha"][1]

Dict{Int64,Array} with 2001 entries:
  1704 => [16.3477 37.0298 10.8647; 14.1652 47.6725 17.2248; 19.9781 38.6197 11…
  1760 => [24.9703 41.2176 10.0525; 18.9364 38.3436 55.1938; 16.235 47.4365 325…
  306  => [15.5293 53.4884 19.6628; 9.92182 49.2556 9.91431; … ; 7.04386 45.763…
  1316 => [20.3984 37.9762 17.2052; 17.6515 47.5653 917.575; … ; 24.4959 40.489…
  1090 => [13.4995 51.908 326.316]
  1333 => [20.6153 37.8863 8.16859; 15.9479 60.0621 32.8428; 2.55459 49.1711 34…
  29   => Array{Int64}(undef,0,3)
  1671 => [12.3654 52.6153 116.788]
  1131 => [5.11092 33.9826 14.491; 21.206 59.335 8.35999; … ; 19.0785 37.1608 1…
  74   => Array{Int64}(undef,0,3)
  1662 => [28.0013 59.8089 6.42274; 32.6915 59.7867 7.17315; 11.8493 51.7938 16…
  1265 => [2.69373 49.2631 45.2905; 14.963 7.53014 13.7126; … ; 3.39943 23.2279…
  905  => [10.8466 21.086 16.2744; 16.6771 52.1715 512.154; … ; 14.7927 15.7681…
  176  => [13.4021 47.6185 27.6766]
  1469 => [20.5529 9.09652 23.0337; 11.0822 21.2638 9.09425

In [21]:
freq=read(arx["freq"])

17.85550205219098